In [1]:
import pandas as pd
import numpy as np
import time
from MyNLP import Divider

In [2]:
# データの読みこみ
comment_list = []
comment_sentence_list = []
df1 = pd.read_csv("comment_keitaiso_0~99_2022_09_22.csv", encoding="Shift-jis")
df2 = pd.read_csv("comment_keitaiso_100~199_2022_09_22.csv", encoding="Shift-jis")
df3 = pd.read_csv("comment_keitaiso_200~299_2022_09_22.csv", encoding="Shift-jis")
df4 = pd.read_csv("comment_keitaiso_300~397_2022_09_22.csv", encoding="Shift-jis")
df5 = pd.read_csv("comment_keitaiso_398~459_2022_09_22.csv", encoding="Shift-jis")

comment_sentence_list.extend(df1.to_numpy().tolist())
comment_sentence_list.extend(df2.to_numpy().tolist())
comment_sentence_list.extend(df3.to_numpy().tolist())
comment_sentence_list.extend(df4.to_numpy().tolist())
comment_sentence_list.extend(df5.to_numpy().tolist())
comment_sentence_list = pd.DataFrame(comment_sentence_list)

# コメントの列だけを抽出
comment_sentence_list = comment_sentence_list.iloc[:, 1]

# 欠損値のデータを削除
comment_sentence_list = comment_sentence_list.dropna()
# dfをリストに変換
comment_sentence_list = comment_sentence_list.tolist()

In [3]:
# Tf-idfベクトルの作成

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(comment_sentence_list)
#print(vectorizer.vocabulary_)
# print(tfidf.toarray())
#print(tfidf)
# print(vectorizer.inverse_transform(tfidf[1]))

In [4]:
tfidf.shape

(1052, 2108)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 閾値以上の類似度である文のペアを保存するプログラム
pair_list = []
pair_list_1 = []
pair_list_2 = []

for id1, comment1 in enumerate(tfidf):
    for id2, comment2 in enumerate(tfidf):
        if not id1 == id2:
            similarity = cosine_similarity(comment1, comment2)
            if similarity > 0.5:
                if not [id2, id1, comment_sentence_list[id2], comment_sentence_list[id1], similarity[0][0]] in pair_list:
                    pair_list.append([id1, id2, comment_sentence_list[id1], comment_sentence_list[id2], similarity[0][0]])
                    print(f"{id1} and {id2} = {similarity}")
            elif similarity > 0.2 and similarity < 0.5:
                if not [id2, id1, comment_sentence_list[id2], comment_sentence_list[id1], similarity[0][0]] in pair_list_1:
                    pair_list_1.append([id1, id2, comment_sentence_list[id1], comment_sentence_list[id2], similarity[0][0]])
                    print(f"{id1} and {id2} = {similarity}")
            else:
                if not [id2, id1, comment_sentence_list[id2], comment_sentence_list[id1], similarity[0][0]] in pair_list_2:
                    pair_list_2.append([id1, id2, comment_sentence_list[id1], comment_sentence_list[id2], similarity[0][0]])
                    # print(f"{id1} and {id2} = {similarity}")

0 and 797 = [[0.29583947]]
0 and 931 = [[0.31282257]]
1 and 123 = [[0.21572087]]
1 and 505 = [[0.25577089]]
1 and 761 = [[0.23197102]]
2 and 99 = [[0.21419029]]
2 and 123 = [[0.29444334]]
2 and 321 = [[0.20391943]]
2 and 456 = [[0.32300654]]
2 and 555 = [[0.21371149]]
2 and 631 = [[0.25686924]]
2 and 673 = [[0.25686924]]
2 and 771 = [[0.20110117]]
2 and 802 = [[0.28878632]]
2 and 980 = [[0.28395823]]
2 and 998 = [[0.20083855]]
2 and 1020 = [[0.247817]]
3 and 22 = [[0.31106823]]
3 and 140 = [[0.25780423]]
3 and 545 = [[0.24129935]]
3 and 1002 = [[0.57923272]]
4 and 417 = [[0.23934465]]
4 and 487 = [[0.23282552]]
4 and 539 = [[0.21282836]]
4 and 745 = [[0.21493527]]
5 and 374 = [[0.21662608]]
5 and 450 = [[0.28805172]]
5 and 545 = [[0.40089191]]
6 and 65 = [[0.20028783]]
6 and 79 = [[0.3103875]]
6 and 180 = [[0.33034596]]
6 and 408 = [[0.24212897]]
6 and 550 = [[0.2219958]]
6 and 682 = [[0.24820869]]
6 and 697 = [[0.24419432]]
6 and 761 = [[0.22175419]]
6 and 781 = [[0.22379218]]
6 and 9

In [ ]:
import csv
with open('comment_similarity_0.5~.csv', 'w', newline='') as csv_file:
    fieldnames = ['id1','id2', 'comment1', 'comment2', 'similarity']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for pair in pair_list:
        data = {'id1': pair[0],'id2': pair[1], 'comment1': pair[2], 'comment2': pair[3], 'similarity': pair[4]}
        try:
            writer.writerow(data)
        except UnicodeEncodeError:
            continue

In [ ]:
import csv
with open('comment_similarity_0.2~0.5.csv', 'w', newline='') as csv_file:
    fieldnames = ['id1','id2', 'comment1', 'comment2', 'similarity']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for pair in pair_list_1:
        data = {'id1': pair[0],'id2': pair[1], 'comment1': pair[2], 'comment2': pair[3], 'similarity': pair[4]}
        try:
            writer.writerow(data)
        except UnicodeEncodeError:
            continue

In [9]:
import csv
with open('comment_similarity_0~0.2.csv', 'w', newline='') as csv_file:
    fieldnames = ['id1','id2', 'comment1', 'comment2', 'similarity']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for pair in pair_list_2:
        data = {'id1': pair[0],'id2': pair[1], 'comment1': pair[2], 'comment2': pair[3], 'similarity': pair[4]}
        try:
            writer.writerow(data)
        except UnicodeEncodeError:
            continue